In [1]:
import pandas as pd
import geopandas
import zipfile
from tqdm.notebook import tqdm

In [2]:
input_spatial_path = "../data/raw/flow/referentiel-comptages-routiers.shp"
input_data_path = "../data/raw/flow/opendata_txt_2019.zip"

daily_output_path = "../data/daily_unmatched_flow.csv"
hourly_output_path = "../data/hourly_unmatched_flow.csv"
    
if "snakemake" in locals():
    input_spatial_path = snakemake.input["spatial"]
    input_data_path = snakemake.input["data"]

    daily_output_path = snakemake.output["daily"]
    hourly_output_path = snakemake.output["hourly"]

In [3]:
df_flow = []

with zipfile.ZipFile(input_data_path) as archive:
    for path in tqdm(archive.namelist()):
        df_local = pd.read_csv(archive.open(path), sep = ";", usecols = [
            "iu_ac", "t_1h", "q"
        ], dtype = {
            "iu_ac": int, "q": float
        })

        df_local = df_local.rename(columns = {
            "iu_ac": "reference_id",
            "t_1h": "timestamp",
            "q": "flow"
        })

        df_local["timestamp"] = pd.to_datetime(df_local["timestamp"])
        df_local["date"] = df_local["timestamp"].dt.date
        df_local["hour"] = df_local["timestamp"].dt.hour
        df_local["is_weekend"] = df_local["timestamp"].dt.weekday >= 5

        df_local = df_local[["reference_id", "date", "is_weekend", "hour", "flow"]]
        df_flow.append(df_local)
        
df_flow = pd.concat(df_flow)

  0%|          | 0/63 [00:00<?, ?it/s]

In [4]:
# Drop entries with NaN flow
df_flow.dropna(inplace = True)

In [5]:
# Drop weekend
df_flow = df_flow[~df_flow["is_weekend"]]
df_flow = df_flow.drop(columns = ["is_weekend"])

In [6]:
# Count how many hours are avaialble for every day and only keep those days where 24h are present
df_hours = df_flow.groupby(["reference_id", "date"]).size().reset_index(name = "hours")
df_hours = df_hours[df_hours["hours"] == 24]

df_flow = pd.merge(df_flow, df_hours, on = ["reference_id", "date"])
df_flow = df_flow.drop(columns = ["hours"])

In [7]:
# Sum up and then average to get daily values
df_daily = df_flow[[
    "reference_id", "date", "flow"
]].groupby(["reference_id", "date"]).sum().reset_index()

df_daily = df_daily[["reference_id", "flow"]].groupby("reference_id").mean().reset_index()

In [8]:
df_daily.to_csv(daily_output_path, sep = ";", index = False)

In [9]:
# Average by hour to get hourly reference
df_hourly = df_flow[[
    "reference_id", "hour", "flow"
]].groupby(["reference_id", "hour"]).mean().reset_index()

In [10]:
df_hourly.to_csv(hourly_output_path, sep = ";", index = False)